# Cloud sandbox powered by FIWARE

## About the open source GPL3 license and copyright for this product

Copyright (c) 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Switch to the Python 3.9 Kernel

When opening this Jupyter Notebook in VSCode, you will need to switch to the Jupyter 3.9 Kernel. 

# Deploy FIWARE and Smart Village microservices

A working cloud project that is capable of receiving data from IoT smart devices and processing edge device data is composed of several important microservices. We will deploy each of the required microservices below to begin sending, receiving, and analyzing IoT edge device data. Then in later notebooks, we will review each microservice and how it works inside of the Smart Village Platform. 

## Deploy the RabbitMQ in the OpenShift Developer Sandbox
A message broker like RabbitMQ can receive messages from smart devices and forward them on to an IoT Agent to process the message. 

Create a RabbitMQ secret. 

In [ ]:
%%bash
oc create secret generic rabbitmq \
  --from-literal rabbitmq-password="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 15)" \
  --from-literal rabbitmq-erlang-cookie="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 15)"
echo DONE

Deploy RabbitMQ with helm. 

In [ ]:
%%bash
helm upgrade --install rabbitmq oci://registry-1.docker.io/bitnamicharts/rabbitmq \
  --set "replicaCount=1" \
  --set "resources.limits.memory=1Gi" \
  --set "resources.limits.cpu=1" \
  --set "resources.requests.cpu=200m" \
  --set "resources.requests.memory=200Mi" \
  --set "auth.username=user" \
  --set "auth.existingPasswordSecret=rabbitmq" \
  --set "auth.existingErlangSecret=rabbitmq" \
  --set "extraPlugins='rabbitmq_auth_backend_ldap rabbitmq_auth_backend_http'" \
  --set "persistence.enabled=false" \
  --set podSecurityContext.enabled=false \
  --set containerSecurityContext.enabled=false
echo DONE

### View RabbitMQ pod details
After deploying RabbitMQ with helm, it will take a minute before the RabbitMQ pod is up and running. Run the command below until the RabbitMQ pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc get pod -l app.kubernetes.io/instance=rabbitmq
oc wait pod -l app.kubernetes.io/instance=rabbitmq --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/instance=rabbitmq
echo DONE

### View RabbitMQ pod logs
If your RabbitMQ pod does not reach the STATUS Running, you can run the command below to view the pod logs of RabbitMQ and check for other errors that may have occured. 

In [ ]:
%%bash
oc logs -l app.kubernetes.io/name=rabbitmq
echo DONE

## Next...
If you have successfully ran all of the commands above, congratulations, you are ready to move on to the next notebook in the course. 

- If you have additional questions or issues, please [create an issue for the course here](https://github.com/computate-org/computate/issues). 
- Otherwise, please continue to the next document [07-deploy-scorpiobroker.ipynb](07-deploy-scorpiobroker.ipynb). 
